<h1>What am I drinking next?</h1>

I wanted to try to use machine learning to predict what bottle of wine I'm going to open next. This is just for laughs and a fun excercise.

Data was acquired from [CellarTracker.com](http://cellartracker.com/), a free web page and an app where I have a database of more than 1700 consumed wines from a decade or so. As I always choose a bottle based on what we are  having for dinner, my assumption is that there is some underlying periodicity to this: one might be more inclined to prepare something you haven't had for a while and choosing the wine accordingly, or one is usually doing more barbeque when it's summer influencing a selection of wine etc.

I'm not (usually) opening the bottles by myself. We have people coming over, we're visiting friends etc. Still, most of the time, I'm usually responsible for selecting a wine in our household.

Okay! Let's first import the necessary libraries:

In [ ]:
!pip install pandas
!pip install scikit-learn

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

Let's load the data and see what we are having:

In [3]:
filename = "My Consumed Bottles.csv"
df = pd.read_csv(filename, header=0, dtype="object")
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1747 entries, 0 to 1746
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Type       1747 non-null   object
 1   Consumed   1747 non-null   object
 2   ShortType  1747 non-null   object
 3   Vintage    1747 non-null   object
 4   Color      1747 non-null   object
 5   Category   1747 non-null   object
 6   Varietal   1747 non-null   object
 7   Country    1747 non-null   object
dtypes: object(8)
memory usage: 109.3+ KB
None


- "Type" is the type of the wine (red, white, sparkling, etc.)
- "Consumed" is the date of the entry
- "ShortType" is the 'type' of the consumption
- "Vintage" is the vintage of the wine
- "Color" is the color of the wine (a bit redundant in this case)
- "Category" is telling if the wine is dry, sweet or such
- "Varietal" is the grape or a blend of the wine
- "Country" is the country where the wine is from

There were some other columns as well, but I omitted them already before exporting.

"Varietal" is the feature we are aiming to predict. There are some overlapping categories I think we can combine:

In [4]:
print(df["Varietal"].unique())

['Cabernet Sauvignon' 'Nebbiolo' 'Champagne Blend' 'Corvina Blend'
 'SuperTuscan Blend' 'Grenache' 'Sangiovese' 'Pinot Grigio'
 'Sangiovese Blend' 'Red Bordeaux Blend' 'Pinot Nero' 'Aglianico' 'Malbec'
 'Red Rhone Blend' 'Malbec Blend' 'Ros� Blend' 'Brachetto'
 'Carignan Blend' 'Bovale' "Nero d'Avola" 'Riesling' 'Moscato' 'Red Blend'
 'White Rhone Blend' 'Tempranillo Blend' 'Grenache Blend' 'Pinot Bianco'
 'Primitivo' 'Zibibbo' 'Merlot' 'Barbera' 'Sauvignon Blanc' 'Zinfandel'
 'Menc�a' 'Port Blend' 'Corvina' 'Bual' 'Chardonnay' 'Nerello Blend'
 'Garganega' 'Cabernet Franc' 'S�millon-Sauvignon Blanc Blend' 'Garnacha'
 'Shiraz' 'White Blend' 'Sauvignon' 'Palomino Fino' 'Pinot Noir'
 'Nerello Mascalese' 'Albari�o' 'Tempranillo' 'Chenin Blanc' 'Arinto'
 'Moscatel' 'Gamay' 'Blauburgunder' 'Sp�tburgunder' 'Gew�rztraminer'
 'Carricante' 'Nascetta' 'Silvaner' 'Cabernet Jura' 'Teran'
 'Sandanski Misket' 'Syrah' 'Vermentino' 'Rubin' 'Furmint Blend' 'Kisi'
 'Nasco' 'Friulano' 'Mourv�dre Blend' 'G

There seems to be some issues with unidentified fonts as well. Let's combine some of the blends with the single variety by hand and fix the font issues while we're at it:

In [5]:
to_replace = {"Corvina Blend" : "Corvina",
              "Nebbiolo Blend" : "Nebbiolo",
              "Chardonnay Blend" : "Chardonnay",
              "Sp�tburgunder" : "Pinot Noir",
              "Blauburgunder" : "Pinot Noir",
              "Tempranillo Blend" : "Tempranillo",
              "Nerello Blend" : "Nerello Mascalese",
              "Sauvignon" : "Sauvignon Blanc",
              "Grenache" : "Grenache Blend",
              "Red Rhone Blend" : "Grenache Blend",
              "Malbec Blend" : "Malbec",
              "Sangiovese Blend" : "Sangiovese",
              "Primitivo" : "Zinfandel",
              "Ros� Blend" : "Rose Blend",
              "Menc�a" : "Mencia",
              "Mourv�dre Blend" : "Mourvedre Blend",
              "Gew�rztraminer" : "Gewurztraminer",
              "Albari�o" : "Albarino",
              "S�millon-Sauvignon Blanc Blend" : "Semillon-Sauvignon Blanc Blend",
              "Carm�n�re" : "Carmenere",
              "Gr�ner Veltliner" : "Gruner Veltliner",
              "Torront�s" : "Torrontes",
              "Pedro Xim�nez" : "Pedro Ximenez",
              "Cabernet Sauvignon" : "Red Bordeaux Blend"
}

df["Varietal"] = df["Varietal"].replace(to_replace)
print(df["Varietal"].unique())

['Red Bordeaux Blend' 'Nebbiolo' 'Champagne Blend' 'Corvina'
 'SuperTuscan Blend' 'Grenache Blend' 'Sangiovese' 'Pinot Grigio'
 'Pinot Nero' 'Aglianico' 'Malbec' 'Rose Blend' 'Brachetto'
 'Carignan Blend' 'Bovale' "Nero d'Avola" 'Riesling' 'Moscato' 'Red Blend'
 'White Rhone Blend' 'Tempranillo' 'Pinot Bianco' 'Zinfandel' 'Zibibbo'
 'Merlot' 'Barbera' 'Sauvignon Blanc' 'Mencia' 'Port Blend' 'Bual'
 'Chardonnay' 'Nerello Mascalese' 'Garganega' 'Cabernet Franc'
 'Semillon-Sauvignon Blanc Blend' 'Garnacha' 'Shiraz' 'White Blend'
 'Palomino Fino' 'Pinot Noir' 'Albarino' 'Chenin Blanc' 'Arinto'
 'Moscatel' 'Gamay' 'Gewurztraminer' 'Carricante' 'Nascetta' 'Silvaner'
 'Cabernet Jura' 'Teran' 'Sandanski Misket' 'Syrah' 'Vermentino' 'Rubin'
 'Furmint Blend' 'Kisi' 'Nasco' 'Friulano' 'Mourvedre Blend'
 'Gruner Veltliner' 'Carmenere' 'Torrontes' 'Pedro Ximenez' 'Arneis'
 'Nero di Troia' 'Dolcetto']


Let's only choose wines that we have actually drank (instead of used for cooking or gifted, for example) and convert the date to Pandas DateTime-object.

In [6]:
df = df[df["ShortType"] == "Drank"]
df["Consumption date"] = pd.to_datetime(df.loc[:,"Consumed"], dayfirst=True)

Then some feature engineering!

1) Let's create a new feature "Month" from "Consumption date".

2) We can remove duplicate entries. These aren't faulty per se as sometimes one opens two bottles of the same wine when there are more people sharing the wine. However, this shouldn't affect the selection of the wine itself!

In [7]:
df["Month"] = df["Consumption date"].dt.month
df = df.drop_duplicates()

3) It is also a bit pointless to include varieties that have appeared only once. Having just one entry does not give any insight to predicting when one might open it next time.

In [8]:
all_varietals = df["Varietal"].value_counts()
singles = all_varietals[all_varietals == 1]
df = df.drop(df[df["Varietal"].isin(singles.index)].index)
varietals = df["Varietal"].unique()

At this point we might drop the unnecessary features and sort the data based on consumption date. Let's also check what we have so far.

In [9]:
df = df.drop(columns=["ShortType", "Type", "Vintage", "Color", "Category", "Country", "Consumed"])
df = df.sort_values(by="Consumption date", ascending=True)
df.reset_index(drop=True, inplace=True)
print(df.head())

             Varietal Consumption date  Month
0          Sangiovese       2016-03-02      3
1               Syrah       2016-03-08      3
2  Red Bordeaux Blend       2016-03-15      3
3  Red Bordeaux Blend       2016-03-16      3
4               Syrah       2016-03-18      3


Let's add some important features to answer some questions of interest:

1) What day of the week it is? This is a big one, obviously, as most of the wines are opened in weekends!

2) How many days it has been since the last time any wine was opened. Could this affect the selection?

3) What was the wine last opened? One might assume that you don't often open the same variety of wine consecutively.

In [10]:
df["Weekday"] = df["Consumption date"].dt.weekday
df["Gap"] = df["Consumption date"].diff(periods=1).dt.days
df["Previous"] = df["Varietal"].shift(periods=1)

Okay, the next one is a bit tedious but it is at the core of the 'theory' I'm trying to prove here: As a selection of wine is based on the food we are having, and if one is more inclined to prepare foods one hasn't had in a while, then an important feature might be the last time a particular wine was consumed.

For this we need to create a new feature for each varietal in our data counting the days since the last time the particular varietal was opened. Here we need to be careful not to introduce any 'data leaking' by accidentaly letting our model know which wine we are opening by setting the number of days to zero too early.

For each varietal 'var', we first add a date when the wine was last opened. Then we use bfill() to fill in the gaps for rows with other varietals. For this to work we need to add one 'dummy' as the last row in the data which is then later removed. Also, np.nan is inserted to rows before the first time the particular varietal is introduced as no previous date is available. Lastly, the date is converted to number of days between the current date and the previous date.

In [11]:
for var in varietals:
    df = pd.concat([df, df.tail(1)], ignore_index=True)
    df.loc[df.index[-1], "Varietal"] = var

    df[var] = df[df["Varietal"] == var]["Consumption date"].shift(periods=1)
    df[var] = df[var].bfill()

    df = df.drop(index=df.index[-1], axis=0)

    first = df[df["Varietal"] == var].index[0]
    df.loc[:first, var] = np.nan

    df[var] = (df["Consumption date"] - df[var]).dt.days

For now, let's choose to only use data from the last couple of years. Older entries include much more random testing and is propably less 'food dependent'. If we're trying to find some patterns here, this might help. After this, we don't need the "Consumption date" anymore.

In [12]:
df = df[df["Consumption date"] > "2020-01-01"]
df = df.drop(columns=["Consumption date"])

Next we can proceed to modeling! 

We use "Varietal" as our target and other columns as features. As "Previous"-column consist of varietals of wines consumed previously, we need to transform this to numerical data. Let's used get_dummies() to do this for us.

The data is also splitted into training and validation sets. I wanna use roughly the last year for testing, as one might think that more mature selections might inlcude more coherent choices of wines.

In [13]:
X = df.drop(columns=["Varietal"])
y = df["Varietal"]

X = pd.get_dummies(X, columns=["Previous"])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


Let's try Random Forest Classifier as it can handle the missing values we have in our columns counting the days since previous consumptions. Then we do the fitting and calculate the accuracy.

In [14]:
model = RandomForestClassifier()

model.fit(X_train, y_train)

print(model.score(X_test, y_test))

0.27472527472527475


<h2>Conclusions</h2>

The prediction gives us an accuracy of roughly 25 - 30 %.

But is this any good?

For practical purposes - not really. If the model can make a right guess roughly one out of a four times, I might still want to select the wines myself in the future. Then again, considering the amount of different varietals available in the data, this doesn't sound too bad either!

But here's the truth: we can easily check the numbers of different varietals in the data:


In [15]:
print((y.value_counts()/y.value_counts().sum())[:5])

Varietal
Corvina               0.205072
Red Bordeaux Blend    0.109151
Sangiovese            0.089305
Grenache Blend        0.071665
Nebbiolo              0.049614
Name: count, dtype: float64


One can see that about 20 % of the consumed bottles have been of "Corvina"-varietal. So just by guessing this on each of the entries without any complicated machine learning nonsense, one would have already gotten accuracy of 20 %.

It is also interesting to see which of the features had most weight in our predictions:

In [16]:
feature_importances = list(zip(model.feature_names_in_, model.feature_importances_))
feature_importances = sorted(feature_importances, key=lambda i: i[1], reverse=True)
for i in feature_importances[:5]:
     print(i[0], np.round(i[1],5))

Gap 0.03051
Corvina 0.02977
Weekday 0.02876
Red Bordeaux Blend 0.02581
Grenache Blend 0.02574


Though the differences aren't huge, we can see that the most important feature is the one counting days since the last time a bottle of "Corvina" was opened.

I can see this being the case as, more often than not, we're having pizza on Saturdays and Corvina is the main grape in the Valpolicella region of Italy - my 'go to' -apellation with pizza for sure!

Let's see if we can confirm this from the data:

In [17]:
print(df[df["Varietal"] == "Corvina"]["Weekday"].value_counts())

Weekday
5    116
6     33
4     16
3      9
0      6
1      4
2      2
Name: count, dtype: int64


Yep, Saturday seems to be the day.

We might play more with different machine learning algorithms but I think the ground truth remains: my habbits of opening wine bottles are too random to make any real predictions for the future based on past behaviour - except for the fact I knew already: Saturday is the pizza day and calls for a nice bottle of Valpolicella.

Salute!